In [20]:
from matplotlib import pyplot as plt
import numpy as np
from netCDF4 import Dataset as NetCDFFile
import math
from scipy.spatial import distance
from math import cos,sin,asin,sqrt
import random

In [2]:
grid = NetCDFFile('/home/vanlaar/nns_python/NarvalDom2_NestE-R02B14_DOM03.nc')
clon_grid = grid.variables['clon'][:]
clat_grid = grid.variables['clat'][:]

In [59]:
def haversine(x, y):
    radius = 6371000
    lon1, lat1 = x
    lon2, lat2 = y

    dlat = lat2-lat1
    dlon = lon2-lon1

    a = sin(dlat/2)**2 + cos(lat1)*cos(lat2)*(sin(dlon/2))**2
    c = 2*asin(sqrt(a))
    distance = radius*c  #distance in m
    return distance



In [4]:
radius = 500

In [18]:
def get_neighbours(ref_cell, radius):
    """ Returns the indeces of the cells that are within radius form the reference cell 
    """

    R = 6378.1 #Radius of the Earth
    d = radius/1000. #Distance in km

    maxlat_brng = np.deg2rad(0)
    maxlon_brng = np.deg2rad(90)
    minlat_brng = np.deg2rad(180)
    minlon_brng = np.deg2rad(270)

    lat1 = clat_grid[ref_cell]
    lon1 = clon_grid[ref_cell]

    maxlat = math.asin( math.sin(lat1)*math.cos(d/R) + math.cos(lat1)*math.sin(d/R)*math.cos(maxlat_brng))
    maxlon = lon1 + math.atan2(math.sin(maxlon_brng)*math.sin(d/R)*math.cos(lat1), math.cos(d/R)-math.sin(lat1)*math.sin(lat1))
    minlat = math.asin( math.sin(lat1)*math.cos(d/R) + math.cos(lat1)*math.sin(d/R)*math.cos(minlat_brng))
    minlon = lon1 + math.atan2(math.sin(minlon_brng)*math.sin(d/R)*math.cos(lat1), math.cos(d/R)-math.sin(lat1)*math.sin(lat1))

    # Gives you a square box around the reference cell:
    cells_box, = np.where((clon_grid < maxlon) & (clon_grid > minlon) & (clat_grid < maxlat) & (clat_grid > minlat))
        
    cellcentres = np.vstack((clon_grid[cells_box],clat_grid[cells_box])).T
    middlecell = np.array([[lon1,lat1]])

    # Compute distance for every point to the reference cell...
    distances = distance.cdist(middlecell, cellcentres, haversine)

    # ...and select those that are inside the given radius to obtain a circle around the reference cell.
    dist_idx = np.where(distances < d*1000)
    cells_circle = cells_box[dist_idx[1]]

    return cells_circle




neighbour_cell = cells[neighbour_coords]
        if neighbour_cell is not None:
            # This cell is occupied: store this index of the contained point    .
            neighbours.append(neighbour_cell)



In [16]:
cell_index = 150
size = 10

cells_circle = get_neighbours(cell_index,300)
print cells_circle


[  148   149   150   151   152 10730 10732 10733]


In [68]:
def point_valid(pt):
    """Is pt a valid point to emit as a sample?

    It must be no closer than r from any other point: check the cells in its
    immediate neighbourhood.

    """
    
    neighbours = get_neighbours(pt, 1000) # radius must be 2*lmax
    for idx in neighbours:
        nearby_pt = idx
        
        # Squared distance between or candidate point, pt, and this nearby_pt.
        #x = np.vstack((clon_grid[pt],clat_grid[pt])).T
        x = clon_grid[pt],clat_grid[pt]
        #y = np.vstack((clon_grid[nearby_pt],clat_grid[nearby_pt])).T
        y = clon_grid[nearby_pt],clat_grid[nearby_pt]
        distance = haversine(x,y)
        if distance < radius:
            # The points are too close, so pt is not a candidate.
            return False
    # All points tested: if we're here, pt is valid
    return True



In [70]:
def get_point(ref_cell, k):
    """Try to find a candidate cell relative to ref_cell to emit in the sample.

    We draw up to k points from the annulus of inner radius r, outer radius 2r
    around the reference point, refpt. If none of them are suitable (because
    they're too close to existing points in the sample), return False.
    Otherwise, return the pt.

    """
    i = 0
    while i < k:
        possibilities = get_neighbours(ref_cell, radius)
        pt = np.random.choice(possibilities)
        print 'in get_point, pt:',pt
        if point_valid(pt):
            print 'valid point:',pt
            return pt
        i += 1
        
    # We failed to find a suitable point in the vicinity of refpt.
    return False

#test = get_point(150, 5)

In [73]:
ncells_grid = len(clon_grid)
ncells_range = np.arange(0,ncells_grid,1)
#random_cells = random.sample(ncells_range, nclouds)

cells = {index: None for index in ncells_range}

# Pick a random point to start with.
pt = np.random.choice(ncells_range)
print 'pt:',pt
samples = [pt]
# Our first sample is indexed at 0 in the samples list...
cells[pt] = 0
# ... and it is active, in the sense that we're going to look for more points
# in its neighbourhood.
active = [0]

nsamples = 1
# As long as there are points in the active list, keep trying to find samples.
while active:
    # choose a random "reference" point from the active list.
    idx = np.random.choice(active)
    print 'idx:',idx
    refpt = samples[idx]
    print 'refpt:',refpt
    # Try to pick a new point relative to the reference point.
    pt = get_point(refpt, 5)
    if pt:
        # Point pt is valid: add it to the samples list and mark it as active
        samples.append(pt)
        nsamples += 1
        active.append(len(samples)-1)
        cells[get_cell_coords(pt)] = len(samples) - 1
    else:
        # We had to give up looking for valid points near refpt, so remove it
        # from the list of "active" points.
        active.remove(idx)
        
print nsamples

pt: 2552632
idx: 0
refpt: 2552632
in get_point, pt: 2552631
in get_point, pt: 2552633
in get_point, pt: 2552634
in get_point, pt: 2552635
in get_point, pt: 2552628
1
